# Chapter 7. Manager Bookmarks

## 7.1 Introduction

### 7.1.1 Layout

### 7.1.2 Edit and Delete Bookmarks

### 7.1.3 Tagging

### 7.1.4 Two categories of advanced things

(1) Templates (Jinja2)

* filters 
* include 
* context processor 
* JavaScript integration

(2) Database (sqlalchemy)

* Many-to-many relation
* Database migration

## 7.2 Demo: Fixing the bookmark layout

(1) Install the flask extension package `flask-moment`.

```bash
$ pip install flask-moment
```

(2) Create a new template `bookmark_list.html` to be included by other htmls for displaying the bookmark list.

(3) In `base.html`, include the JavaScript libraries via `flask-moment`.

(4) Define some customized sytles in `main.css`.

In [ ]:
# SAVE AS __init__.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from flask.ext.script import Manager, prompt_bool

from thermos import app, db
from thermos.models import User

manager = Manager(app)

@manager.command
def initdb():
    db.create_all()
    db.session.add(User(username='reindert', email='reindert@example.com', password='test'))
    db.session.add(User(username='arjen', email='arjen@example.com', password='test'))
    db.session.commit()
    print('Initialized the database')
    
@manager.command
def dropdb():
    if prompt_bool(
        'Are you sure you want to lose all your data'):
        db.drop_all()
        print('Dropped the database')
        
if __name__ == '__main__':
    manager.run()

## 7.3 Demo: Editing bookmarks

(1) Move `add.html` to `bookmark_form.html` for both adding and editing bookmarks.

(2) Add the view function for editing a bookmark and modify the view function for adding a bookmark in `views.py`.

In [ ]:
# SAVE AS views.py

# -*- coding: utf-8 -*-

from flask import render_template, flash, redirect, url_for, request, abort
from flask_login import login_required, login_user, logout_user, current_user

from . import app, db, login_manager
from thermos.forms import BookmarkForm, LoginForm, SignupForm
from thermos.models import User, Bookmark 
    
@login_manager.user_loader
def load_user(userid):
    return User.query.get(int(userid))

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=Bookmark.newest(5))
                           
@app.route('/add', methods=['GET', 'POST'])
@login_required
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        bm = Bookmark(user=current_user, url=url, description=description)
        db.session.add(bm)
        db.session.commit()
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('index'))
    return render_template('bookmark_form.html', form=form, title='Add a bookmark')
    
@app.route('/edit/<int:bookmark_id>', methods=['GET', 'POST'])
@login_required
def edit_bookmark(bookmark_id):
    bookmark = Bookmark.query.get_or_404(bookmark_id)
    if current_user != bookmark.user:
        abort(403)
    form = BookmarkForm(obj=bookmark)
    if form.validate_on_submit():
        form.populate_obj(bookmark)
        db.session.commit()
        flash("Stored '{}'".format(bookmark.description))
        return redirect(url_for('user', username=current_user.username))
    return render_template('bookmark_form.html', form=form, title='Edit bookmark')  
  
@app.route('/user/<username>')
def user(username):
    user = User.query.filter_by(username=username).first_or_404()
    return render_template('user.html', user=user)
   
@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        # login and validate the user...
        user = User.get_by_username(form.username.data)
        if user is not None and user.check_password(form.password.data):
            login_user(user, form.remember_me.data)
            flash('Logged in successfully as {}'.format(user.username))
            return redirect(request.args.get('next') or url_for('user', username=user.username))
        flash('Incorrect username or password.')
    return render_template('login.html', form=form)
    
@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('index'))
    
@app.route('/signup', methods=['GET', 'POST'])
def signup():
    form = SignupForm()
    if form.validate_on_submit():
        user = User(email=form.email.data,
                    username=form.username.data,
                    password=form.password.data)
        db.session.add(user)
        db.session.commit()
        flash('Welcome, {}! Please login'.format(user.username))
        return redirect(url_for('login'))
    return render_template('signup.html', form=form)

@app.errorhandler(403)
def permission_not_allowed(e):
    return render_template('403.html'), 403
        
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500  

## 7.4 Review: Jinja2 filters and populating objects

### 7.4.1 Jinja2 filters

(1) Filters are special functions

* applied in a variable expression
* can be chained using the pipe symbol (`|`)

(2) `{{ title|truncate(10) }}`

Will truncate the contents of the variable title

(3) `{{ title|trim|upper|truncate(10) }}`

Strip whitespace from title, then convert to uppercase, then truncate

(4) Jinja provides built-in filters for many common operations.

* Can also add custom filters with `app.template_filter` decorator
* Flask adds one: `tojson`

### 7.4.2 Jinja2: Include and With

(1) `{% include "other.html" % }`

* renders `other.html` and includes the output in current template
* useful with templates that generate HTML fragments
* included template has access to context variables

(2) `{% with x=42 %}`

creates a new scope with variable `x` in it

### 7.4.3 Populating forms and models

(1) `form = BookmarkForm(obj=bookmark)`

Use the `obj` argument to populate a form with data from an object

* For example a Model instance
* If request data is present, that will be used instead of obj

(2) `form.populate_obj(bookmark)`

Use `populate_obj` to populate a Model instance from an object

## 7.5 Demo: Database migrations with `flask-migrate`

(1) For demo purpose, delete the old database file `thermos.db`.

```bash
$ rm thermos.db
```

(2) Install the Flask extension package `flask-migrate` for the automatic database migration.

```bash
$ pip install flask-migrate
```

(3) Create the migrator in `manager.py`.

In [ ]:
# SAVE AS manager.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from thermos import app, db
# Although Bookmark is not referenced in this file, it is needed to imported so that 
# it can be detected by the database migrator.
from thermos.models import User, Bookmark 

from flask.ext.script import Manager, prompt_bool
from flask.ext.migrate import Migrate, MigrateCommand

manager = Manager(app)
migrate = Migrate(app, db)

manager.add_command('db', MigrateCommand)

@manager.command
def initdb():
    db.create_all()
    db.session.add(User(username='reindert', email='reindert@example.com', password='test'))
    db.session.add(User(username='arjen', email='arjen@example.com', password='test'))
    db.session.commit()
    print('Initialized the database')
    
@manager.command
def dropdb():
    if prompt_bool(
        'Are you sure you want to lose all your data'):
        db.drop_all()
        print('Dropped the database')
        
if __name__ == '__main__':
    manager.run()

(4) Show all the available migrate commands.

```bash
$ python manager.py db
usage: Perform database migrations

Perform database migrations

positional arguments:
  {init,revision,migrate,edit,merge,upgrade,downgrade,show,history,heads,branches,current,stamp}
    init                Creates a new migration repository
    revision            Create a new revision file.
    migrate             Alias for 'revision --autogenerate'
    edit                Edit current revision.
    merge               Merge two revisions together. Creates a new migration
                        file
    upgrade             Upgrade to a later version
    downgrade           Revert to a previous version
    show                Show the revision denoted by the given symbol.
    history             List changeset scripts in chronological order.
    heads               Show current available heads in the script directory
    branches            Show current branch points
    current             Display the current revision for each database.
    stamp               'stamp' the revision table with the given revision;
                        don't run any migrations

optional arguments:
  -?, --help            show this help message and exit
```

(5) Set up the migrator.

```bash
$ python manager.py db init
  Creating directory /home/renwei/repos/github/learning-ml/python/pluralsight-intro2flask/thermos/migrations ... done
  Creating directory /home/renwei/repos/github/learning-ml/python/pluralsight-intro2flask/thermos/migrations/versions ... done
  Generating /home/renwei/repos/github/learning-ml/python/pluralsight-intro2flask/thermos/migrations/script.py.mako ... done
  Generating /home/renwei/repos/github/learning-ml/python/pluralsight-intro2flask/thermos/migrations/README ... done
  Generating /home/renwei/repos/github/learning-ml/python/pluralsight-intro2flask/thermos/migrations/alembic.ini ... done
  Generating /home/renwei/repos/github/learning-ml/python/pluralsight-intro2flask/thermos/migrations/env.py ... done
  Please edit configuration/connection/logging settings in '/home/renwei/repos/github/learning-ml/python/pluralsight-
  intro2flask/thermos/migrations/alembic.ini' before proceeding.
```

(6) Generate an upgrade script with the name `initial` for the first upgrade.

```bash
$ python manager.py db upgrade -m "initial"
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.autogenerate.compare] Detected added table 'user'
INFO  [alembic.autogenerate.compare] Detected added table 'bookmark'
  Generating /home/renwei/repos/github/learning-ml/python/pluralsight-intro2flask/thermos/migrations/versions/298d693908ab_initial.py ... done
```

(7) Do the first upgrade.

```bash
$ python manager.py db upgrade
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 298d693908ab, initial
```

If you browse the database file `thermos.db`, you will see a new table `alembic_version` for recording the database version.

## 7.6 Demo: A many-to-many relation

(1) Create a junction table in `models.py` for the many-to-many relation between tag and bookmark.

In [ ]:
# SAVE AS models.py

# -*- coding: utf-8 -*-

from datetime import datetime

from sqlalchemy import desc
from flask_login import UserMixin
from werkzeug.security import check_password_hash, generate_password_hash

from . import db

# Create a junction table which connects the foreign keys of two tables 
# Bookmark and Tag
tags = db.Table('bookmark_tag',
    db.Column('tag_id', db.Integer, db.ForeignKey('tag.id')),
    db.Column('bookmark_id', db.Integer, db.ForeignKey('bookmark.id')))

class Bookmark(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    url = db.Column(db.Text, nullable=False)
    # Pass the function object instead of the function result 
    # as the default method for getting the default time.
    date = db.Column(db.DateTime, default=datetime.utcnow)  
    description = db.Column(db.String(300))
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    _tags = db.relationship('Tag', secondary=tags, 
                            backref=db.backref('bookmarks', lazy='dynamic'))
    
    @staticmethod
    def newest(num):
        return Bookmark.query.order_by(desc(Bookmark.date)).limit(num)
    
    def __repr__(self):
        return "<Bookmark '{}': '{}'>".format(self.description, self.url)

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True)
    email = db.Column(db.String(120), unique=True)
    bookmarks = db.relationship('Bookmark', backref='user', lazy='dynamic')
    password_hash = db.Column(db.String)
    
    @property
    def password(self):
        raise AttributeError('password: write-only field')
        
    @password.setter
    def password(self, password):
        self.password_hash = generate_password_hash(password)
        
    def check_password(self, password):
        return check_password_hash(self.password_hash, password)
        
    @staticmethod
    def get_by_username(username):
        return User.query.filter_by(username=username).first()
    
    def __repr__(self):
        return '<User %r>' % self.username
        
class Tag(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(25), nullable=False, unique=True, index=True)
    
    def __repr__(self):
        return self.name

(2) Modify `manager.py` to detect the new junction table. Also change the command `initdb` into `insert_data` to insert some data for testing.

In [ ]:
# SAVE AS manager.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from thermos import app, db
# Although Bookmark is not referenced in this file, it is needed to imported so that 
# it can be detected by the database migrator.
from thermos.models import User, Bookmark, Tag

from flask.ext.script import Manager, prompt_bool
from flask.ext.migrate import Migrate, MigrateCommand

manager = Manager(app)
migrate = Migrate(app, db)

manager.add_command('db', MigrateCommand)

@manager.command
def insert_data():
    reindert = User(username="reindert", email="reindert@example.com", password="test")
    db.session.add(reindert)

    def add_bookmark(url, description, tags):
        db.session.add(Bookmark(url=url, description=description, user=reindert,
                                tags=tags))

    for name in ["python", "flask", "webdev", "programming", "training", "news", "orm", "databases", "emacs", "gtd", "django"]:
        db.session.add(Tag(name=name))
    db.session.commit()

    add_bookmark("http://www.pluralsight.com", "Pluralsight. Hardcore developer training.", "training,programming,python,flask,webdev")
    add_bookmark("http://www.python.org", "Python - my favorite language", "python")
    add_bookmark("http://flask.pocoo.org", "Flask: Web development one drop at a time.", "python,flask,webdev")
    add_bookmark("http://www.reddit.com", "Reddit. Frontpage of the internet", "news,coolstuff,fun")
    add_bookmark("http://www.sqlalchemyorg", "Nice ORM framework", "python,orm,databases")

    arjen = User(username="arjen", email="arjen@robben.nl", password="test")
    db.session.add(arjen)
    db.session.commit()
    print 'Initialized the database'
    
@manager.command
def dropdb():
    if prompt_bool(
        'Are you sure you want to lose all your data'):
        db.drop_all()
        print('Dropped the database')
        
if __name__ == '__main__':
    manager.run()

(3) Generate a new upgrade script with the name `tags`.

```
$ python manager.py db migrate -m "tags
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.autogenerate.compare] Detected added table 'tag'
INFO  [alembic.autogenerate.compare] Detected added index 'ix_tag_name' on '['name']'
INFO  [alembic.autogenerate.compare] Detected added table 'bookmark_tag'
  Generating /home/renwei/repos/github/learning-ml/python/pluralsight-intro2flask/thermos/migrations/versions/fba10e8f81f9_tags.py ... done
```

(4) Do the upgrade.

```bash
$ python manager.py db upgrade
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade 298d693908ab -> fba10e8f81f9, tags
```

or 

```bash
$ python manager.py upgrade --tag tags
```

(5) If you want to downgrade the database,

```bash
$ python manager.py downgrade
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running downgrade fba10e8f81f9 -> 298d693908ab, tags
```

or 

```bash
$ python manager.py downgrade --tag initial
```

## 7.7 Review: Flask-Migrate

Database migrations

(1) `pip install flask-migrate`

(2) In `manager.py`:

```python
from flask.ext.migrate import Migrate, MigrateCommand
migrate = Migrate(app, db)
manager.add_command('db', MigrateCommand)
```

(3) `python manager.py db init`

Initializes migrations and only needs to run once.

(4) `python manager.py db migrate -m "migration_name"`

Adds a migration - generate scripts

(5) `python manager.py db upgrade`

* Upgrade database to latest version
* Actually runs SQL

## 7.8 Demo: Forms and views for tags

(1) Modify the bookmark form.

In [ ]:
# SAVE AS forms.py

# -*- coding: utf-8 -*-

from flask_wtf import Form
from wtforms.fields import StringField, PasswordField, BooleanField, SubmitField
from flask.ext.wtf.html5 import URLField
from wtforms.validators import DataRequired, Length, Email, Regexp, EqualTo,\
    url, ValidationError
    
from thermos.models import User

class BookmarkForm(Form):
    url = URLField('The URL for your bookmark', validators=[DataRequired(), url()])
    description = StringField('Add an optional description:')
    tags = StringField('Tags', validators=[Regexp(r'^[a-zA-Z0-9, ]*$',
                    message='Tags can only contain letters and numbers')])
    
    def validate(self):
        if not (self.url.data.startswith("http://") or\
            self.url.data.startswith("https://")):
            self.url.data = "http://" + self.url.data
            
        if not Form.validate(self):
            return False
            
        if not self.description.data:
            self.description.data = self.url.data
            
        # filter out empty and duplicate tag names
        stripped = [t.strip() for t in self.tags.data.split(',')]
        not_empty = [tag for tag in stripped if tag]
        tagset = set(not_empty)
        self.tags.data = ','.join(tagset)
            
        return True
        
class LoginForm(Form):
    username = StringField('Your Username', validators=[DataRequired()])
    password = PasswordField('Password', validators=[DataRequired()])
    remember_me = BooleanField('Keep me logged in')
    submit = SubmitField('Log In')
    
class SignupForm(Form):
    username = StringField('Username',
                    validators=[
                        DataRequired(), Length(3, 80),
                        Regexp('^[A-Za-z0-9_]{3,}$',
                            message='Usernames consist of numbers, letters, '
                                    'and underscores.')])
    password = PasswordField('Password',
                    validators=[
                        DataRequired(),
                        EqualTo('password2', message='Passwords must match')])
    password2 = PasswordField('Confirm password', validators=[DataRequired()])
    email = StringField('Email',
                        validators=[DataRequired(), Length(1, 80), Email()])
                        
    def validate_email(self, email_field):
        if User.query.filter_by(email=email_field.data).first():
            raise ValidationError('There already is a user with this email address.')
            
    def validate_username(self, username_field):
        if User.query.filter_by(username=username_field.data).first():
            raise ValidationError('This username is already taken.')

(2) Modify `bookmark_form.html` and `bookmark_list.html`.

(3) Modify `models.py`.

In [ ]:
# SAVE AS models.py

# -*- coding: utf-8 -*-

from datetime import datetime

from sqlalchemy import desc
from flask_login import UserMixin
from werkzeug.security import check_password_hash, generate_password_hash

from . import db

# Create a junction table which connects the foreign keys of two tables 
# Bookmark and Tag
tags = db.Table('bookmark_tag',
    db.Column('tag_id', db.Integer, db.ForeignKey('tag.id')),
    db.Column('bookmark_id', db.Integer, db.ForeignKey('bookmark.id')))

class Bookmark(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    url = db.Column(db.Text, nullable=False)
    # Pass the function object instead of the function result 
    # as the default method for getting the default time.
    date = db.Column(db.DateTime, default=datetime.utcnow)  
    description = db.Column(db.String(300))
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    _tags = db.relationship('Tag', secondary=tags, 
                            backref=db.backref('bookmarks', lazy='dynamic'))
    
    @staticmethod
    def newest(num):
        return Bookmark.query.order_by(desc(Bookmark.date)).limit(num)
        
    @property
    def tags(self):
        return ",".join([t.name for t in self._tags])
        
    @tags.setter
    def tags(self, string):
        if string:
            self._tags = [Tag.get_or_create(name) for name in string.split(',')]
    
    def __repr__(self):
        return "<Bookmark '{}': '{}'>".format(self.description, self.url)

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True)
    email = db.Column(db.String(120), unique=True)
    bookmarks = db.relationship('Bookmark', backref='user', lazy='dynamic')
    password_hash = db.Column(db.String)
    
    @property
    def password(self):
        raise AttributeError('password: write-only field')
        
    @password.setter
    def password(self, password):
        self.password_hash = generate_password_hash(password)
        
    def check_password(self, password):
        return check_password_hash(self.password_hash, password)
        
    @staticmethod
    def get_by_username(username):
        return User.query.filter_by(username=username).first()
    
    def __repr__(self):
        return '<User %r>' % self.username
        
class Tag(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(25), nullable=False, unique=True, index=True)
    
    @staticmethod
    def get_or_create(name):
        try:
            return Tag.query.filter_by(name=name).one()
        except:
            return Tag(name=name)
             
    def __repr__(self):
        return self.name

(4) Insert some test data into the database.

```bash
$ python manager.py insert_data
```

(5) Run the app and test the tag features.

```bash
$ python manager.py runserver
```

## 7.9 Integrating Javascript

To improve the bookmark tag input box, e.g., show a list of suggestions, we need to use Javascript.

(1) Download the Javascript library `select2`.

https://github.com/select2/select2/releases

Note that **we have to download a relatively old version 3.5.0**, otherwise the UI enhancement won't work. We have tried 3.5.1 and 4.0.5, and neither works with the code given by the instructor.

(2) Copy a couple of files (mainly `select2.min.js`) to our project.

(3) Update `base.html` and `bookmark_form.html` to use `select2`.

## 7.10 Demo: A context processor

(1) `all_tags()` is not passed as usuall as a view argument and it is in fact a context processor.

In [ ]:
# SAVE AS models.py

# -*- coding: utf-8 -*-

from datetime import datetime

from sqlalchemy import desc
from flask_login import UserMixin
from werkzeug.security import check_password_hash, generate_password_hash

from . import db

# Create a junction table which connects the foreign keys of two tables 
# Bookmark and Tag
tags = db.Table('bookmark_tag',
    db.Column('tag_id', db.Integer, db.ForeignKey('tag.id')),
    db.Column('bookmark_id', db.Integer, db.ForeignKey('bookmark.id')))

class Bookmark(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    url = db.Column(db.Text, nullable=False)
    # Pass the function object instead of the function result 
    # as the default method for getting the default time.
    date = db.Column(db.DateTime, default=datetime.utcnow)  
    description = db.Column(db.String(300))
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    _tags = db.relationship('Tag', secondary=tags, 
                            backref=db.backref('bookmarks', lazy='dynamic'))
    
    @staticmethod
    def newest(num):
        return Bookmark.query.order_by(desc(Bookmark.date)).limit(num)
        
    @property
    def tags(self):
        return ",".join([t.name for t in self._tags])
        
    @tags.setter
    def tags(self, string):
        if string:
            self._tags = [Tag.get_or_create(name) for name in string.split(',')]
    
    def __repr__(self):
        return "<Bookmark '{}': '{}'>".format(self.description, self.url)

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True)
    email = db.Column(db.String(120), unique=True)
    bookmarks = db.relationship('Bookmark', backref='user', lazy='dynamic')
    password_hash = db.Column(db.String)
    
    @property
    def password(self):
        raise AttributeError('password: write-only field')
        
    @password.setter
    def password(self, password):
        self.password_hash = generate_password_hash(password)
        
    def check_password(self, password):
        return check_password_hash(self.password_hash, password)
        
    @staticmethod
    def get_by_username(username):
        return User.query.filter_by(username=username).first()
    
    def __repr__(self):
        return '<User %r>' % self.username
        
class Tag(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(25), nullable=False, unique=True, index=True)
    
    @staticmethod
    def get_or_create(name):
        try:
            return Tag.query.filter_by(name=name).one()
        except:
            return Tag(name=name)
           
    @staticmethod
    def all():
        return Tag.query.all()
             
    def __repr__(self):
        return self.name

In [ ]:
# SAVE AS views.py

# -*- coding: utf-8 -*-

from flask import render_template, flash, redirect, url_for, request, abort
from flask_login import login_required, login_user, logout_user, current_user

from . import app, db, login_manager
from thermos.forms import BookmarkForm, LoginForm, SignupForm
from thermos.models import User, Bookmark, Tag
    
@login_manager.user_loader
def load_user(userid):
    return User.query.get(int(userid))

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=Bookmark.newest(5))
                           
@app.route('/add', methods=['GET', 'POST'])
@login_required
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        tags = form.tags.data
        bm = Bookmark(user=current_user, url=url, description=description, tags=tags)
        db.session.add(bm)
        db.session.commit()
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('index'))
    return render_template('bookmark_form.html', form=form, title='Add a bookmark')
    
@app.route('/edit/<int:bookmark_id>', methods=['GET', 'POST'])
@login_required
def edit_bookmark(bookmark_id):
    bookmark = Bookmark.query.get_or_404(bookmark_id)
    if current_user != bookmark.user:
        abort(403)
    form = BookmarkForm(obj=bookmark)
    if form.validate_on_submit():
        form.populate_obj(bookmark)
        db.session.commit()
        flash("Stored '{}'".format(bookmark.description))
        return redirect(url_for('user', username=current_user.username))
    return render_template('bookmark_form.html', form=form, title='Edit bookmark')  
  
@app.route('/user/<username>')
def user(username):
    user = User.query.filter_by(username=username).first_or_404()
    return render_template('user.html', user=user)
   
@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        # login and validate the user...
        user = User.get_by_username(form.username.data)
        if user is not None and user.check_password(form.password.data):
            login_user(user, form.remember_me.data)
            flash('Logged in successfully as {}'.format(user.username))
            return redirect(request.args.get('next') or url_for('user', username=user.username))
        flash('Incorrect username or password.')
    return render_template('login.html', form=form)
    
@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('index'))
    
@app.route('/signup', methods=['GET', 'POST'])
def signup():
    form = SignupForm()
    if form.validate_on_submit():
        user = User(email=form.email.data,
                    username=form.username.data,
                    password=form.password.data)
        db.session.add(user)
        db.session.commit()
        flash('Welcome, {}! Please login'.format(user.username))
        return redirect(url_for('login'))
    return render_template('signup.html', form=form)

@app.errorhandler(403)
def forbidden(e):
    return render_template('403.html'), 403
        
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500
    
@app.context_processor
def inject_tags():
    return dict(all_tags=Tag.all)

(2) Add a tag view and create a new template `tag.html`.

In [ ]:
# SAVE AS views.py

# -*- coding: utf-8 -*-

from flask import render_template, flash, redirect, url_for, request, abort
from flask_login import login_required, login_user, logout_user, current_user

from . import app, db, login_manager
from thermos.forms import BookmarkForm, LoginForm, SignupForm
from thermos.models import User, Bookmark, Tag
    
@login_manager.user_loader
def load_user(userid):
    return User.query.get(int(userid))

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=Bookmark.newest(5))
                           
@app.route('/add', methods=['GET', 'POST'])
@login_required
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        tags = form.tags.data
        bm = Bookmark(user=current_user, url=url, description=description, tags=tags)
        db.session.add(bm)
        db.session.commit()
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('index'))
    return render_template('bookmark_form.html', form=form, title='Add a bookmark')
    
@app.route('/edit/<int:bookmark_id>', methods=['GET', 'POST'])
@login_required
def edit_bookmark(bookmark_id):
    bookmark = Bookmark.query.get_or_404(bookmark_id)
    if current_user != bookmark.user:
        abort(403)
    form = BookmarkForm(obj=bookmark)
    if form.validate_on_submit():
        form.populate_obj(bookmark)
        db.session.commit()
        flash("Stored '{}'".format(bookmark.description))
        return redirect(url_for('user', username=current_user.username))
    return render_template('bookmark_form.html', form=form, title='Edit bookmark')  
  
@app.route('/user/<username>')
def user(username):
    user = User.query.filter_by(username=username).first_or_404()
    return render_template('user.html', user=user)
   
@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        # login and validate the user...
        user = User.get_by_username(form.username.data)
        if user is not None and user.check_password(form.password.data):
            login_user(user, form.remember_me.data)
            flash('Logged in successfully as {}'.format(user.username))
            return redirect(request.args.get('next') or url_for('user', username=user.username))
        flash('Incorrect username or password.')
    return render_template('login.html', form=form)
    
@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('index'))
    
@app.route('/signup', methods=['GET', 'POST'])
def signup():
    form = SignupForm()
    if form.validate_on_submit():
        user = User(email=form.email.data,
                    username=form.username.data,
                    password=form.password.data)
        db.session.add(user)
        db.session.commit()
        flash('Welcome, {}! Please login'.format(user.username))
        return redirect(url_for('login'))
    return render_template('signup.html', form=form)
    
@app.route('/tag/<name>')
def tag(name):
    tag = Tag.query.fitler_by(name=name).first_or_404()
    return render_template('tag.html', tag=tag)

@app.errorhandler(403)
def forbidden(e):
    return render_template('403.html'), 403
        
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500
    
@app.context_processor
def inject_tags():
    return dict(all_tags=Tag.all)

(3) Also add the links to the tags in `base.html` and `bookmark_list.html`.

(4) Add some light-color styling for displaying the tags in the sidebar.

## 7.11 Review: Context processors and super

(1) Inject variables into template context.

* A function that returns a dict
* Will be run before each template is loaded
* Contents of dict are added to template context
* Annotate with `@app.context_processor`.

```python
@app.context_processor
def inject_tags():
    return dict(all_tags=Tag.all)
```

Note that we pass `Tag.all` as a function object not `Tag.all()` as invoking the function object. In this case, each template will have the capability of invoking `Tag.all()` but won't execute `Tag.all()` unless it explicitly does so.

(2) Calling `super()` inside a `{% block %}`

Will insert contents of a parent block

## 7.12 Demo: A delete page

(1) Add a route for `delete` in `views.py`.

In [ ]:
# SAVE AS views.py

# -*- coding: utf-8 -*-

from flask import render_template, flash, redirect, url_for, request, abort
from flask_login import login_required, login_user, logout_user, current_user

from . import app, db, login_manager
from thermos.forms import BookmarkForm, LoginForm, SignupForm
from thermos.models import User, Bookmark, Tag
    
@login_manager.user_loader
def load_user(userid):
    return User.query.get(int(userid))

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=Bookmark.newest(5))
                           
@app.route('/add', methods=['GET', 'POST'])
@login_required
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        tags = form.tags.data
        bm = Bookmark(user=current_user, url=url, description=description, tags=tags)
        db.session.add(bm)
        db.session.commit()
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('index'))
    return render_template('bookmark_form.html', form=form, title='Add a bookmark')
    
@app.route('/edit/<int:bookmark_id>', methods=['GET', 'POST'])
@login_required
def edit_bookmark(bookmark_id):
    bookmark = Bookmark.query.get_or_404(bookmark_id)
    if current_user != bookmark.user:
        abort(403)
    form = BookmarkForm(obj=bookmark)
    if form.validate_on_submit():
        form.populate_obj(bookmark)
        db.session.commit()
        flash("Stored '{}'".format(bookmark.description))
        return redirect(url_for('user', username=current_user.username))
    return render_template('bookmark_form.html', form=form, title='Edit bookmark')
    
@app.route('/delete/<int:bookmark_id>', methods=['GET', 'POST'])
@login_required
def delete_bookmark(bookmark_id):
    bookmark = Bookmark.query.get_or_404(bookmark_id)
    if current_user != bookmark.user:
        abort(403)
    if request.method == 'POST':
        db.session.delete(bookmark)
        db.session.commit()
        flash("Delete '{}'".format(bookmark.description))
        return redirect(url_for('user', username=current_user.username))
    else:
        flash('Please confirm deleting the bookmark.')
    return render_template('confirm_delete.html', bookmark=bookmark, nolinks=True)
  
@app.route('/user/<username>')
def user(username):
    user = User.query.filter_by(username=username).first_or_404()
    return render_template('user.html', user=user)
   
@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        # login and validate the user...
        user = User.get_by_username(form.username.data)
        if user is not None and user.check_password(form.password.data):
            login_user(user, form.remember_me.data)
            flash('Logged in successfully as {}'.format(user.username))
            return redirect(request.args.get('next') or url_for('user', username=user.username))
        flash('Incorrect username or password.')
    return render_template('login.html', form=form)
    
@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('index'))
    
@app.route('/signup', methods=['GET', 'POST'])
def signup():
    form = SignupForm()
    if form.validate_on_submit():
        user = User(email=form.email.data,
                    username=form.username.data,
                    password=form.password.data)
        db.session.add(user)
        db.session.commit()
        flash('Welcome, {}! Please login'.format(user.username))
        return redirect(url_for('login'))
    return render_template('signup.html', form=form)
    
@app.route('/tag/<name>')
def tag(name):
    tag = Tag.query.filter_by(name=name).first_or_404()
    return render_template('tag.html', tag=tag)

@app.errorhandler(403)
def forbidden(e):
    return render_template('403.html'), 403
        
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500
    
@app.context_processor
def inject_tags():
    return dict(all_tags=Tag.all)

(2) Create a new template page `confirm_delete.html`. Also move the part of displaying a single bookmark into `bookmark.html` so that it can be included by `confirm_delete.html` and `bookmark_list.html`

(3) Modify the bookmark styling in `main.css`.

(4) Add a delete link in `bookmark.html`.

## 7.13 Resources and summary

### 7.13.1 Resources

(1) `flask-migrate`

http://flask-migrate.readthedocs.org

(2) `flask-moment`: the Flask integration of the Javascript library `moment`

https://github.com/miguelgrinberg/flask-moment/

(3) `select2`

https://ivaynberg.github.io/select2/

(3) Jinja2 built-in filters:

http://jinja.pocoo.org

### 7.13.2 Summary

(1) Layout

(2) Templates: filters, includes

(3) Editing bookmarks

Populating models from forms, and forms from models

(4) Tags

* Many-to-many
* `flask-migrate`
* Javascript
* Context processor

(5) Delete page